### Использование моделей
Для отбора признаков чаще всего используют L1 регуляризацию с линейной/логистической регрессией или случайный лес (о котором вы пока не знаете)

In [44]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

# Еще раз покажем результат с простой моделью и исходными данными

lr = LogisticRegression()
print(cross_val_score(lr, x_data_generated, y_data_generated, scoring='accuracy').mean())

0.9093137254901961


In [45]:
# Теперь произведем отбор признаков с помощью нашей логистической регрессии,
# Отобранные признаки подадим далее в саму логистическую регрессию

pipe = make_pipeline(SelectFromModel(estimator=LogisticRegression(penalty='l1',)), LogisticRegression())
print(cross_val_score(pipe, x_data_generated, y_data_generated, scoring='accuracy').mean())

0.8890931372549019


### Перебор признаков

* **Exhaustive Feature Selection** - полный перебор всех возможных сочетаний и подмножеств признаков. Очень долго, но точно найдет нужное.

* **SequentialFeatureSelector** - найдет лучшее подмножество из k признаков.

In [46]:
# устанавливаем mlxtend:
!pip install mlxtend  #))))))horay, i don't have to work hard))))))

Invalid requirement: '#))))))horay,'
Traceback (most recent call last):
  File "C:\Users\Lain\Anaconda3\lib\site-packages\pip\_vendor\packaging\requirements.py", line 93, in __init__
    req = REQUIREMENT.parseString(requirement_string)
  File "C:\Users\Lain\Anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 1632, in parseString
    raise exc
  File "C:\Users\Lain\Anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 1622, in parseString
    loc, tokens = self._parse( instring, 0 )
  File "C:\Users\Lain\Anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 1379, in _parseNoCache
    loc,tokens = self.parseImpl( instring, preloc, doActions )
  File "C:\Users\Lain\Anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 3395, in parseImpl
    loc, exprtokens = e._parse( instring, loc, doActions )
  File "C:\Users\Lain\Anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 1379, in _parseNoCache
    loc,tokens = self.parseImpl( instring, preloc, doActions )
  

In [47]:
from mlxtend.feature_selection import SequentialFeatureSelector

selector = SequentialFeatureSelector(LogisticRegression(), scoring='accuracy',
                                     verbose=2, k_features=3, forward=False, n_jobs=-1)

selector.fit(x_data_generated, y_data_generated)

[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    2.6s finished

[2018-10-12 19:03:45] Features: 19/3 -- score: 0.9399999999999998[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:    2.6s finished

[2018-10-12 19:03:48] Features: 18/3 -- score: 0.95[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:    2.6s finished

[2018-10-12 19:03:51] Features: 17/3 -- score: 0.95[Parallel(n_jobs=-1)]: Done  17 out of  17 | elapsed:    2.7s finished

[2018-10-12 19:03:54] Features: 16/3 -- score: 0.9400000000000001[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:    2.6s finished

[2018-10-12 19:03:57] Features: 15/3 -- score: 0.9400000000000001[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    2.6s finished

[2018-10-12 19:04:00] Features: 14/3 -- score: 0.95[Parallel(n_jobs=-1)]: Done  14 out of  14 | elapsed:    2.5s finished

[2018-10-12 19:04:03] Features: 13/3 -- score: 0.95[Parallel(n_jobs=-1)]: Done  13 out of  13 | elapsed:    2.1s remaining:    0.0s
[Parallel(n_jobs

SequentialFeatureSelector(clone_estimator=True, cv=5,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
             floating=False, forward=False, k_features=3, n_jobs=-1,
             pre_dispatch='2*n_jobs', scoring='accuracy', verbose=2)

Итак, удалось достигнуть accuracy=0.99, что супер круто! Но это, конечно, синтетическая задачка. С реальными данными все намного сложнее.

# KaggleInClass

Начнем решать соревнование на платформе `Kaggle` (пока только `InClass`)

Ссылка на соревнование: https://www.kaggle.com/c/best-home

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
'''from sklearn.feature_selection import VarianceThreshold
from sklearn.datasets import make_classification
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline #smth like inearRegression - but it can also look at results of other models
from sklearn.linear_model import LogisticRegression
import warnings
from scipy.stats import lognorm
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import reverse_geocoder as revgc
from sklearn import preprocessing'''

'from sklearn.feature_selection import VarianceThreshold\nfrom sklearn.datasets import make_classification\nfrom sklearn.preprocessing import PolynomialFeatures\nfrom sklearn.pipeline import Pipeline #smth like inearRegression - but it can also look at results of other models\nfrom sklearn.linear_model import LogisticRegression\nimport warnings\nfrom scipy.stats import lognorm\nfrom sklearn.preprocessing import MinMaxScaler\nfrom sklearn.preprocessing import StandardScaler\nimport reverse_geocoder as revgc\nfrom sklearn import preprocessing'

In [2]:
df = pd.read_csv("all/train.csv")
dtest = pd.read_csv("all/test.csv")

mas = ['building_id','created','display_address','features','manager_id','photos']
df = df.drop(mas,axis=1)
X_test = dtest.drop(mas,axis=1)
X= df.drop(['TARGET'],axis = 1)
df['TARGET']=df['TARGET'].replace(['low','medium','high'],[-1,0,1])
y = df['TARGET']

In [3]:
Id_column = X_test['Id']
X_test.head()

,Id,bathrooms,bedrooms,latitude,listing_id,longitude,price
0,25364,3.0,3,40.7695,7082228,-73.9603,7300
1,26511,1.0,2,40.7067,6980308,-74.0101,2650
2,27100,2.0,4,40.7272,7221448,-73.9859,6200
3,10404,1.0,3,40.7701,6854758,-73.9550,4858
4,7654,1.0,1,40.7878,6871682,-73.9763,401


In [4]:
#высчитываем mean для каждого стобца используя и x, и x_test
means = [0 for i in range(len(X.columns))]
for i in range(len(X.columns)):
    means[i] = (X[X.columns[i]].dropna().mean()+X_test[df.columns[i]].dropna().mean())/2
    #X[X.columns[i]].replace(np.nan,means[i])
    X_test[X_test.columns[i]].replace(np.nan,means[i])
    row = X[X.columns[i]]
    row_t = X_test[X_test.columns[i]]
    X[X.columns[i]] = (row - row.mean()) / row.std()
    X_test[X_test.columns[i]] = (row_t - row_t.mean()) / row_t.std()
X= X.dropna(subset = X.columns.values)
print(means)

[24749.343545685417, 1.2117644674987842, 1.5427811098503268, 40.740183191495035, 7023650.204233533, -73.95453719673684, 3780.74654184903]


In [5]:
X.head()

,Id,bathrooms,bedrooms,latitude,listing_id,longitude,price
0,-0.462817,-0.424255,-1.381203,0.161932,1.555236,0.017573,-0.096489
1,0.191266,-0.424255,-1.381203,0.060282,-0.465383,-0.021865,-0.059145
2,1.321244,-0.424255,-0.484608,-0.028081,-1.528432,-0.019588,-0.038704
3,-0.532226,-0.424255,-0.484608,0.074566,0.583114,-0.017675,-0.028680
4,-0.769506,-0.424255,0.411987,-0.063459,0.655613,-0.046184,0.062164


In [6]:
for i in range(len(X.columns)):
    for j in range(i,len(X.columns)):
        X[str(X.columns[i])+'+'+str(X.columns[j])] = X[X.columns[i]]*X[X.columns[j]]

In [77]:
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression
selector = SequentialFeatureSelector(LogisticRegression(), scoring='neg_mean_squared_error',
                                     verbose=2, k_features=3, forward=False, n_jobs=-1)

selector.fit(X, y)

JoblibTypeError: JoblibTypeError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\Lain\Anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...nda3\\lib\\site-packages\\ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
C:\Users\Lain\Anaconda3\lib\runpy.py in 

In [78]:
from sklearn.linear_model import Ridge

reg = Ridge().fit(X, y)

# Посмотрим на коэффициенты модели
# Коэффициенты
print("Сдвиг={}".format(reg.intercept_))
for column, coef in zip(X.columns, reg.coef_):
    print("Коэффициент для {}={}".format(column, coef))
print()

errors = -cross_val_score(Ridge(), X, y, cv=5, scoring='neg_mean_squared_error')
print("Результат={}".format(np.average(errors)))

C:\Users\Lain\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number5.446302e-17
  overwrite_a=True).T


Сдвиг=-0.6771684345742021
Коэффициент для Id=-0.015500433396952966
Коэффициент для bathrooms=0.03481636869738149
Коэффициент для bedrooms=0.17326581899672472
Коэффициент для latitude=-0.5264445585698037
Коэффициент для listing_id=-0.0016665142049206267
Коэффициент для longitude=-0.44956253146133984
Коэффициент для price=-4.122130542487208
Коэффициент для Id+Id=-0.022401347904836236
Коэффициент для Id+bathrooms=0.03419920101787633
Коэффициент для Id+bedrooms=-0.013146469431168232
Коэффициент для Id+latitude=-0.02042113799859442
Коэффициент для Id+listing_id=-0.013272155049667292
Коэффициент для Id+longitude=-0.4719447355600999
Коэффициент для Id+price=-0.1896646412434874
Коэффициент для bathrooms+bathrooms=-0.0358912215978945
Коэффициент для bathrooms+bedrooms=0.00014934145974669367
Коэффициент для bathrooms+latitude=0.24424795173770084
Коэффициент для bathrooms+listing_id=0.016435799369158878
Коэффициент для bathrooms+longitude=-0.03709565962398221
Коэффициент для bathrooms+price=0.919

C:\Users\Lain\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number8.960214e-17
  overwrite_a=True).T
C:\Users\Lain\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number5.925188e-17
  overwrite_a=True).T
C:\Users\Lain\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number5.585407e-17
  overwrite_a=True).T


Результат=418536117.60016453


C:\Users\Lain\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number7.987608e-17
  overwrite_a=True).T


In [ ]:
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from sklearn.model_selection import cross_val_score
tree =  DecisionTreeClassifier(random_state=42)
tree.fit(X,y)
print(np.mean(cross_val_score(DecisionTreeClassifier(random_state=42),X,y,cv = 10)))

In [66]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(X, y)

# Посмотрим на коэффициенты модели
# Коэффициенты
print("Сдвиг={}".format(reg.intercept_))
for column, coef in zip(X.columns, reg.coef_):
    print("Коэффициент для {}={}".format(column, coef))
print()

errors = -cross_val_score(LinearRegression(), X, y, cv=5, scoring='neg_mean_squared_error')
print("Результат={}".format(np.average(errors)))

Сдвиг=-0.6169017128654014
Коэффициент для Id=0.003378697733714042
Коэффициент для bathrooms=-0.0858968313702957
Коэффициент для bedrooms=0.0625143563315352
Коэффициент для latitude=-0.012305373442097782
Коэффициент для listing_id=-0.01679740090249188
Коэффициент для longitude=-0.01369358456637915
Коэффициент для price=-0.011450219294241915

Результат=0.38956880787649484


In [74]:
#X_test = X_test.dropna(subset=X_test.columns.values) 

In [72]:
ans = reg.predict(X_test) 
for i in range(len(ans)):
    ans[i] = int(round(ans[i]))
result = pd.DataFrame({'TARGET':ans,'Id':Id_column})
result['TARGET'] = result['TARGET'].replace([-1,0,1],['low','medium','high'])
result.to_csv('answers.csv',index = False,sep = ',')
#answers = pd.write_csv("all/answers.csv",result)

In [73]:
root = pd.read_csv("answers.csv")
root.head()

,TARGET,Id
0,low,25364
1,low,26511
2,low,27100
3,medium,10404
4,low,7654
